# Youtube API
1. Con la cuenta de developers ir al panel de la biblioteca de API's (https://console.cloud.google.com/apis/library?hl=es-419&authuser=4)
2. Dar click en *seleccionar un proyecto*
3. Dar click en *Proyecto Nuevo*
4. Dar nombre el proyecto "Prueba Python"
5. Dejar *Sin organización*
6. Dar click en crear
7. Buscar en la biblioteca "Youtube Data API v3"
8. Dar click en *Habilitar*
9. Dar click en *CREAR CREDENCIALES*
10. Click en *Datos Públicos* y dejar Youtube API v3
11. Copiar *Clave de API*
o.

In [1]:
#!pip install --upgrade google-api-python-client

In [2]:
from googleapiclient.discovery import build
from apirequest import CallGoogle #Dejare el modulo con las credenciales vacias
from datetime import datetime
google_api_key  = CallGoogle()
youtube = build('youtube','v3',developerKey = google_api_key)

In [4]:
#Vamos a ver las estadisticas de un canal -> Escoge el que tu quieras.
request = youtube.channels().list(
    part='statistics',
    forUsername = 'HatsuneMiku' #Nombre de tu canal que quieras hacer la recopilacion de datos. (Sugerencia: uno de música)
)
response = request.execute()
response

{'kind': 'youtube#channelListResponse',
 'etag': 'VwJAtY92azWpFdbl87mj7PiOAYw',
 'pageInfo': {'totalResults': 1, 'resultsPerPage': 5},
 'items': [{'kind': 'youtube#channel',
   'etag': '_wOaaNyzE9vwtlIj4lYDRkcGt1I',
   'id': 'UCJwGWV914kBlV4dKRn7AEFA',
   'statistics': {'viewCount': '255610712',
    'subscriberCount': '2520000',
    'hiddenSubscriberCount': False,
    'videoCount': '526'}}]}

In [5]:
#Notemos que podemos tener informacion del canal usando su ID
#¿Cómo lo obtenemos? Lo que nos regresó anteriormente es un diccionarios de diccionarios
#Suena enrredado pero mediante el acceso por indices se facilita más las cosas
id_youtube = response['items'][0]['id'] #Accediendo por un ID.
id_youtube

'UCJwGWV914kBlV4dKRn7AEFA'

In [6]:
#Ahora con este id podemos tener sus listas de reproducción
play_list = youtube.playlists().list(
    part = 'contentDetails,snippet', #Datos de contenido de la listas de reproduccion
    channelId = id_youtube,
    maxResults=200 #Maximo resultados. Si hay más de 200 solo mostrará 200 resultados
)
play_list_result = play_list.execute()
play_list_result

{'kind': 'youtube#playlistListResponse',
 'etag': 'odzDzx0QWYVMV0pFD4Um7Ovq4-Q',
 'pageInfo': {'totalResults': 13, 'resultsPerPage': 50},
 'items': [{'kind': 'youtube#playlist',
   'etag': 'ehmx3thLBqRSXXOjzzkUBPbnGs0',
   'id': 'PL-pKPpZ1Q5NacsYBHSI5wyUQlsV8HhKqL',
   'snippet': {'publishedAt': '2023-04-06T04:18:37Z',
    'channelId': 'UCJwGWV914kBlV4dKRn7AEFA',
    'title': 'STUDY WITH MIKU',
    'description': '',
    'thumbnails': {'default': {'url': 'https://i.ytimg.com/vi/gdssxLx7ofs/default.jpg',
      'width': 120,
      'height': 90},
     'medium': {'url': 'https://i.ytimg.com/vi/gdssxLx7ofs/mqdefault.jpg',
      'width': 320,
      'height': 180},
     'high': {'url': 'https://i.ytimg.com/vi/gdssxLx7ofs/hqdefault.jpg',
      'width': 480,
      'height': 360},
     'standard': {'url': 'https://i.ytimg.com/vi/gdssxLx7ofs/sddefault.jpg',
      'width': 640,
      'height': 480},
     'maxres': {'url': 'https://i.ytimg.com/vi/gdssxLx7ofs/maxresdefault.jpg',
      'width': 1280,

In [7]:
#Ahora vamos a obtener todos los resultados de las estas listas o el contenido de ellas
totalPlaylist = []
while play_list is not None: #Simpre y cuando no sean nulas.
    play_list_result = play_list.execute()
    totalPlaylist += play_list_result['items']
    play_list = youtube.playlists().list_next(play_list, play_list_result)

print(f"total: {len(totalPlaylist)}") #Total de listas de reproduccion
totalPlaylist #Contenido -> La salida es una lista de diccionarios.

total: 13


[{'kind': 'youtube#playlist',
  'etag': 'ehmx3thLBqRSXXOjzzkUBPbnGs0',
  'id': 'PL-pKPpZ1Q5NacsYBHSI5wyUQlsV8HhKqL',
  'snippet': {'publishedAt': '2023-04-06T04:18:37Z',
   'channelId': 'UCJwGWV914kBlV4dKRn7AEFA',
   'title': 'STUDY WITH MIKU',
   'description': '',
   'thumbnails': {'default': {'url': 'https://i.ytimg.com/vi/gdssxLx7ofs/default.jpg',
     'width': 120,
     'height': 90},
    'medium': {'url': 'https://i.ytimg.com/vi/gdssxLx7ofs/mqdefault.jpg',
     'width': 320,
     'height': 180},
    'high': {'url': 'https://i.ytimg.com/vi/gdssxLx7ofs/hqdefault.jpg',
     'width': 480,
     'height': 360},
    'standard': {'url': 'https://i.ytimg.com/vi/gdssxLx7ofs/sddefault.jpg',
     'width': 640,
     'height': 480},
    'maxres': {'url': 'https://i.ytimg.com/vi/gdssxLx7ofs/maxresdefault.jpg',
     'width': 1280,
     'height': 720}},
   'channelTitle': 'HatsuneMiku',
   'localized': {'title': 'STUDY WITH MIKU', 'description': ''}},
  'contentDetails': {'itemCount': 2}},
 {'kin

In [8]:
#Si accedemos por indices veremos que esto se puede apreciar más
totalPlaylist[1]['id'] #El id de la segunda lista

'PL-pKPpZ1Q5NYK6AN_5lybs6xc5LlD5Pwj'

In [9]:
#Youtube tiene un formato de tiempo un poco extraño para verlo en la hora normal
#lo que haremos es hacer una función que nos convierta ese horario extraño a 
#uno que podamos ver y entender
def timestream(time):
    time = time.replace("T","").replace("Z","") #Quitamos la T y Z del string
    date,hour = time[0:10],time[11:21] #separamos entre fecha y hora
    time = '{} {}'.format(date ,hour) #Lo concatenamos en una forma diferente
    datetime_object = datetime.strptime(time, "%Y-%m-%d %H:%M:%S") #Aplicamos una funcion de datetime para convertirlo en formato de hora
    return datetime_object #Retornamos la hora del objeto

In [10]:
#En este apartado vamos a obtener varias cosas como:
# ID, titulo de la lista de reproduccion, No. Canciones, Fecha, la imagen de portada de la lista de reproduccion
from datetime import datetime
playlist_ids = []
playlist_name = []
playlist_pictureStandard = []
playlist_timepublish = []
playlist_noSongs = []
for j in range(0,len(totalPlaylist)):
    playlist_ids.append(totalPlaylist[j]['id'])
    playlist_name.append(totalPlaylist[j]['snippet']['title'])
    playlist_pictureStandard.append(totalPlaylist[j]['snippet']['thumbnails']['standard']['url'])
    playlist_timepublish.append(timestream(totalPlaylist[j]['snippet']['publishedAt'])) #Aqui aplicamos nuestra funcion para tener un objeto datetime
    playlist_noSongs.append(totalPlaylist[j]['contentDetails']['itemCount'])

In [22]:
#Ahora aplicando el concocimiento de Pandas generemos un dataFrame con esos elementos
import pandas as pd
dataframe = {'Ids de playlists':playlist_ids,
             'Titulos de playlist':playlist_name,
             'Numero de canciones':playlist_noSongs,
             'Uri':playlist_pictureStandard,
             'Fecha publicacion':playlist_timepublish}
df = pd.DataFrame(data=dataframe)
df

,Ids de playlists,Titulos de playlist,Numero de canciones,Uri,Fecha publicacion
0,PL-pKPpZ1Q5NacsYBHSI5wyUQlsV8HhKqL,STUDY WITH MIKU,2,https://i.ytimg.com/vi/gdssxLx7ofs/sddefault.jpg,2023-04-06 04:18:37
1,PL-pKPpZ1Q5NYK6AN_5lybs6xc5LlD5Pwj,おすすめのニューリリース/Recommended New Releases,16,https://i.ytimg.com/vi/mNq4xWsRsT0/sddefault.jpg,2022-10-21 01:31:46
2,PL-pKPpZ1Q5Nam8i_mRK4Ky0t0-ihdlE2U,【Hatsune Miku/MIKU EXPO】関連ソング/Songs【初音ミク/MIKU ...,18,https://i.ytimg.com/vi/AYUNaQaDfa8/sddefault.jpg,2022-02-08 02:56:45
3,PL-pKPpZ1Q5Nbr1RmMnQ7hcyRo_CXLAOsy,【Hatsune Miku】アニバーサリーソング/Anniversary song【初音ミク】,18,https://i.ytimg.com/vi/JIBFZ7Siz1I/sddefault.jpg,2022-01-20 09:42:41
4,PL-pKPpZ1Q5NbP6N1MrSkCKTCHooY_Jbm7,【Hatsune Miku/MAGICAL MIRAI】関連ソング/Songs 【初音ミク/...,25,https://i.ytimg.com/vi/o4AxMk3SGUY/sddefault.jpg,2022-01-20 09:40:05
5,PL-pKPpZ1Q5Nbw0GsDT9V6aGzrQASzWZ6k,【Hatsune Miku/SNOWMIKU】 テーマソング/Theme song 【初音ミ...,10,https://i.ytimg.com/vi/iksyHmE1JAQ/sddefault.jpg,2022-01-20 09:22:57
6,PL-pKPpZ1Q5NYrZw1fT6jSLCMwQqwPTewZ,【Kagamine Rin/Len】鏡音リン・レン Happy 14th Birthday ...,15,https://i.ytimg.com/vi/6W3MbW_hs4o/sddefault.jpg,2021-12-14 08:17:13
7,PL-pKPpZ1Q5NYXIzQjjo-hH6qrJbnb1yKT,【Hatsune Miku】 Demo Songs 【初音ミク】,52,https://i.ytimg.com/vi/HNVfTmJV3Rw/sddefault.jpg,2017-11-02 02:47:15
8,PL-pKPpZ1Q5NYdsmuLrHLFJg1eL6bXgNDx,【Hatsune Miku】 GSR 【初音ミク】,21,https://i.ytimg.com/vi/IT62xM035hs/sddefault.jpg,2017-11-02 01:13:49
9,PL-pKPpZ1Q5NZqi-pHqc2zKXk8GG3U2fkW,【Hatsune Miku】 CONCERT 【初音ミク】,36,https://i.ytimg.com/vi/jhl5afLEKdo/sddefault.jpg,2013-06-13 08:22:49


In [23]:
#Si nos damos cuenta de algo, es muy simple y es un poco de texto en la Urls
#de las imagenes, podemos hacerlo más presentable convirtiendo el dataFrame en HTML.
from IPython.core.display import HTML
def a_img_tg(url):
    return '<img src="'+ url + '" width="100" >'
display(HTML(df.to_html(escape=False,formatters=dict(Uri=a_img_tg))))

,Ids de playlists,Titulos de playlist,Numero de canciones,Uri,Fecha publicacion
0,PL-pKPpZ1Q5NacsYBHSI5wyUQlsV8HhKqL,STUDY WITH MIKU,2,,2023-04-06 04:18:37
1,PL-pKPpZ1Q5NYK6AN_5lybs6xc5LlD5Pwj,おすすめのニューリリース/Recommended New Releases,16,,2022-10-21 01:31:46
2,PL-pKPpZ1Q5Nam8i_mRK4Ky0t0-ihdlE2U,【Hatsune Miku/MIKU EXPO】関連ソング/Songs【初音ミク/MIKU EXPO】,18,,2022-02-08 02:56:45
3,PL-pKPpZ1Q5Nbr1RmMnQ7hcyRo_CXLAOsy,【Hatsune Miku】アニバーサリーソング/Anniversary song【初音ミク】,18,,2022-01-20 09:42:41
4,PL-pKPpZ1Q5NbP6N1MrSkCKTCHooY_Jbm7,【Hatsune Miku/MAGICAL MIRAI】関連ソング/Songs 【初音ミク/マジカルミライ】,25,,2022-01-20 09:40:05
5,PL-pKPpZ1Q5Nbw0GsDT9V6aGzrQASzWZ6k,【Hatsune Miku/SNOWMIKU】 テーマソング/Theme song 【初音ミク/雪ミク】,10,,2022-01-20 09:22:57
6,PL-pKPpZ1Q5NYrZw1fT6jSLCMwQqwPTewZ,【Kagamine Rin/Len】鏡音リン・レン Happy 14th Birthday #Shorts【鏡音リン・レン】,15,,2021-12-14 08:17:13
7,PL-pKPpZ1Q5NYXIzQjjo-hH6qrJbnb1yKT,【Hatsune Miku】 Demo Songs 【初音ミク】,52,,2017-11-02 02:47:15
8,PL-pKPpZ1Q5NYdsmuLrHLFJg1eL6bXgNDx,【Hatsune Miku】 GSR 【初音ミク】,21,,2017-11-02 01:13:49
9,PL-pKPpZ1Q5NZqi-pHqc2zKXk8GG3U2fkW,【Hatsune Miku】 CONCERT 【初音ミク】,36,,2013-06-13 08:22:49


## Obtengamos las duraciones por cada lista.
En este caso agregaremos un parametro más, el tiempo de duración y hacerlo en un dataframe diferentes

In [11]:
playlists_total = [] #Guardar la información de cada lista de reproducción
for i in range(0,len(playlist_ids)):
    details_playlist = youtube.playlistItems().list(
        part = 'contentDetails',
        playlistId  = playlist_ids[i], #Pedir los detalles por cada lista.
        maxResults=200 #Maximo de resultados.
    )
    playlistIdResult  = details_playlist.execute() #Hacemos la petición a Google.
    playlists_total.append(playlistIdResult) # agregamos los resultados a la lista.

In [12]:
#Imprimamos los resultados.
for i in range(0,len(playlists_total)):
    for item in playlists_total[i]['items']:
        print(item)
    print()

{'kind': 'youtube#playlistItem', 'etag': 'sYgOygN0fqDdv2pC4jVAqIjW5iY', 'id': 'UEwtcEtQcFoxUTVOYWNzWUJIU0k1d3lVUWxzVjhIaEtxTC41NkI0NEY2RDEwNTU3Q0M2', 'contentDetails': {'videoId': 'gdssxLx7ofs', 'videoPublishedAt': '2023-03-02T09:45:02Z'}}
{'kind': 'youtube#playlistItem', 'etag': '91oUO_VSyN3M_7p1sTJeKAZBu5I', 'id': 'UEwtcEtQcFoxUTVOYWNzWUJIU0k1d3lVUWxzVjhIaEtxTC4yODlGNEE0NkRGMEEzMEQy', 'contentDetails': {'videoId': 'J8P_1fVRYQI', 'videoPublishedAt': '2023-05-10T09:45:00Z'}}

{'kind': 'youtube#playlistItem', 'etag': 'POPmR8tYvktuTIlbsDa8wFTy2Tc', 'id': 'UEwtcEtQcFoxUTVOWUs2QU5fNWx5YnM2eGM1TGxENVB3ai4wRUE3MkE2QzM5M0M3RUMx', 'contentDetails': {'videoId': 'mNq4xWsRsT0', 'videoPublishedAt': '2023-08-02T09:00:35Z'}}
{'kind': 'youtube#playlistItem', 'etag': 'gkchFczzCINWqnLsw-X7yUyyYgI', 'id': 'UEwtcEtQcFoxUTVOWUs2QU5fNWx5YnM2eGM1TGxENVB3ai5BNDEyODI2QzIyNkJCRTMx', 'contentDetails': {'videoId': 'ZHvMuYydhBM', 'videoPublishedAt': '2023-08-01T12:00:34Z'}}
{'kind': 'youtube#playlistItem', 'etag'

In [13]:
#Veremos que puede ser un poco grande en tener cada lista de producción y sacar 
#el tiempo de reproducción manualmente por cada lista. Para solucionar este 
#problema usaremos listas vacias en listas vacias, es decir, generaremos listas
#vacias entro de una lista ¿Cuántas listas vacias? Será por cada lista de reproduccion.
#El resultado son listas vacias que almacenaremos los ids de los videos por 
#cada lista de reproducción
rangePlayList = [[] for _ in range(0,len(playlists_total))]
rangePlayList

[[], [], [], [], [], [], [], [], [], [], [], [], []]

In [14]:
#Ahora almacenaremos los elementos en cada lista vacia.
for i in range(0,len(playlists_total)):
    for item in playlists_total[i]['items']:
        vd = item['contentDetails']['videoId']
        if i==i:
            rangePlayList[i].append(vd)
rangePlayList#Lista de ids por lista de reproduccion

[['gdssxLx7ofs', 'J8P_1fVRYQI'],
 ['mNq4xWsRsT0',
  'ZHvMuYydhBM',
  '96Ep3k-29TU',
  'u8XPBVIC1Wc',
  'lvTJe4VvSyc',
  'oFWR2Bf9oKc',
  'zeqTyuE4DZc',
  '5L0jcPn6rsQ',
  'Zwb_vfkJB4A',
  'MsFtXEYwh9s',
  'iyKhpX3RWS8',
  'ZLcjy23sz98',
  'FSXsAt7z_RU',
  '9VeRZk761aw',
  'p0UjmYJ9sfw',
  'R_g0M5Zmqhg'],
 ['AYUNaQaDfa8',
  'gvfhcQ5F22M',
  '3lce-CE50SE',
  'AufydOsiD6M',
  'bMkQnpo_Ifg',
  'eyn9Bvr4UQ4',
  'sK92X82T3Sk',
  'N1-Z8uslIsI',
  'GRc_Vz6kEjc',
  'nf9RjiHG_dY',
  'tY8OhmplZWY',
  'TMdHGvrwyJc',
  'e29aYbcF6_A',
  'tsYOZj9qOHQ',
  'jrBzSRiXuvo',
  'fkgaFnf1Ae8',
  'VudoIo9i_w4',
  'L1mBG5ZA9AU'],
 ['JIBFZ7Siz1I',
  'TQD9fU3mTAA',
  'GMQ87OVLBDA',
  'DJfPFNLVVR8',
  'pvG05Z4J8s0',
  'BgrWfhnYjAk',
  'DDcJhy4F5mo',
  'MLYs1vpOz8c',
  'heTaHWABCOo',
  'XQh5Uf5itkI',
  'OO6xVm2ronw',
  'oYOmZ7a39Qo',
  'n4N3h4iuovs',
  'LrOQMyT4iZY',
  'QlfXsuPEJ8s',
  'oEkGC2HV7rc',
  '6DpudRojTB4',
  'RsmJ--RNgE8'],
 ['o4AxMk3SGUY',
  'IsxdBZ0wgq8',
  'b-xO9poaPEs',
  '7j0mQH0BtEU',
  'ZOTJgXBkJ

In [15]:
#Definamos ahora una función para que sea más efectivo en que nos regrese los
#detalles de cada video por cada lista de reproducción y solamente la llamemos
#cuando sea necesario hacer una petición a Google.
def videos_reponse(id_videos):
    video_request  = youtube.videos().list(
        part='contentDetails',
        id=','.join(id_videos)
    )
    video_data = video_request.execute()
    return video_data #Regresa la información de cada video usando su id

In [16]:
#obtencion de la duracion de los videos->duration
playlist1_videos = videos_reponse(rangePlayList[0]) #En este ejemplo es para la lista 1 y sus videos
playlist2_videos = videos_reponse(rangePlayList[1]) #En este para la lista 2 y sus ejemplos 
for item in playlist1_videos['items']:
    duration = item['contentDetails']['duration']
    print(duration)#Formato de Youtube será la duración que veremos que nos da un formato no reconocible
    print()

PT1H30M1S

PT1H30M1S



In [17]:
#Aquí empezaremos a hablar de Expresiones Regulares 
#Dejaré aquí la documentación correspondiente.
#https://docs.python.org/es/3/howto/regex.html
import re 
hours_re = re.compile(r'(\d+)H')
minuntes_re = re.compile(r'(\d+)M')
seconds_re = re.compile(r'(\d+)S')
for item in playlist1_videos['items']:
    duration = item['contentDetails']['duration']
    horas = hours_re.search(duration)
    minutos = minuntes_re.search(duration)
    segundos = seconds_re.search(duration)
    print(horas,minutos,segundos)
    print()

<re.Match object; span=(2, 4), match='1H'> <re.Match object; span=(4, 7), match='30M'> <re.Match object; span=(7, 9), match='1S'>

<re.Match object; span=(2, 4), match='1H'> <re.Match object; span=(4, 7), match='30M'> <re.Match object; span=(7, 9), match='1S'>



In [18]:
#Cuando ejecutamos la celda de arriba veremos que tenemos en match = 'H'
#minutos y finalmente los segundos pero seguimos teniendo el problema en
#poder visualizarlo de manera más con detalle
#Para ello vamos a arreglar este for usando métodos que si no hay horas 
#o minutos (pueden ser una lista de shorts) regrese un 0 
for item in playlist1_videos['items']:
    duration = item['contentDetails']['duration']
    horas = hours_re.search(duration)
    minutos = minuntes_re.search(duration)
    segundos = seconds_re.search(duration)
    #Si no tenemos horas o minutos extensos
    horas = int(horas.group(1)) if horas else 0
    minutos = int(minutos.group(1)) if minutos else 0
    segundos = int(segundos.group(1)) if segundos else 0
    print(horas,minutos,segundos)
    print()

1 30 1

1 30 1



In [19]:
#Veremos que es un poco más entendible pero ahora no es tan claro.
#Vemos que tenemos que hacer esto por cada lista de reproducción,
#hagamos esto para n-videos. Definiremos una funcion de nos regrese
#el tiempo en formato de tiempo o la reproducción entre los segundos
#totales,es decir, si en una lista hay dos videos de 1hr entonces regre-
#sará 7200 segundos.
from datetime import timedelta #Transformar tiempo a segundos
def timevideosPlaylist(playlist):
    time_videos = []#Tiempo de cada video por cada lista de reproduccion.
    
    for item in playlist['items']:
        duration = item['contentDetails']['duration']
        horas = hours_re.search(duration)
        minutos = minuntes_re.search(duration)
        segundos = seconds_re.search(duration)
        horas = int(horas.group(1)) if horas else 0
        minutos = int(minutos.group(1)) if minutos else 0
        segundos = int(segundos.group(1)) if segundos else 0
        #Por cada video
        video_segundos = timedelta(
            hours = horas,
            minutes = minutos,
            seconds = segundos
        ).total_seconds()
        #Por cada video de la lista de reproduccion
        time_videos.append(video_segundos)
    total_reproduction = int(sum(time_videos))
    minutos,segundos =divmod(total_reproduction,60)
    horas,minutos = divmod(minutos,60)
    total_reproduction = f'{horas}:{minutos}:{segundos}' #Formato H:M:S
    return [time_videos,total_reproduction]

timevideosPlaylist(playlist2_videos) #Tenemos resultados para la segunda lista de reproduccion para cada n-video.

[[207.0,
  302.0,
  128.0,
  215.0,
  160.0,
  187.0,
  206.0,
  188.0,
  108.0,
  234.0,
  179.0,
  176.0,
  117.0,
  220.0,
  322.0,
  220.0],
 '0:52:49']

In [20]:
#Obtengamos la suma de cada lista resproducción con todos los videos usando la
#función que vimos arriba y aplicando esta función que nos resgresará una lista
#y cada elemento será en formato de hora de cada lista de reproducción en el canal
def timeStreamingChannels(listaTotales):
    timeStreaming_seconds = []
    for item in listaTotales:
        plt= videos_reponse(item)
        time = timevideosPlaylist(plt)[1] #Obtenemos el formato de hora.
        timeStreaming_seconds.append(time)
    return timeStreaming_seconds
timeStreamingChannels(rangePlayList) #Lo usamos para toda la lista de reproducciones del canal

['3:0:2',
 '0:52:49',
 '1:13:42',
 '1:9:34',
 '1:43:8',
 '0:41:27',
 '0:4:15',
 '1:47:23',
 '1:15:28',
 '2:50:2',
 '15:47:2',
 '0:27:5',
 '3:13:34']

In [24]:
#Finalmente volveremos a correr el código  y añadiendo una columna donde podamos
#visualizar el tiempod de duración de cada lisa de reproducción en total
duration = timeStreamingChannels(rangePlayList)
df['Duracion'] = duration #Agregamos una nueva columna con los datos al dataFrame
from IPython.core.display import HTML
def a_img_tg(url):
    return '<img src="'+ url + '" width="150" >'
display(HTML(df.to_html(escape=False,formatters=dict(Uri=a_img_tg))))

,Ids de playlists,Titulos de playlist,Numero de canciones,Uri,Fecha publicacion,Duracion
0,PL-pKPpZ1Q5NacsYBHSI5wyUQlsV8HhKqL,STUDY WITH MIKU,2,,2023-04-06 04:18:37,3:0:2
1,PL-pKPpZ1Q5NYK6AN_5lybs6xc5LlD5Pwj,おすすめのニューリリース/Recommended New Releases,16,,2022-10-21 01:31:46,0:52:49
2,PL-pKPpZ1Q5Nam8i_mRK4Ky0t0-ihdlE2U,【Hatsune Miku/MIKU EXPO】関連ソング/Songs【初音ミク/MIKU EXPO】,18,,2022-02-08 02:56:45,1:13:42
3,PL-pKPpZ1Q5Nbr1RmMnQ7hcyRo_CXLAOsy,【Hatsune Miku】アニバーサリーソング/Anniversary song【初音ミク】,18,,2022-01-20 09:42:41,1:9:34
4,PL-pKPpZ1Q5NbP6N1MrSkCKTCHooY_Jbm7,【Hatsune Miku/MAGICAL MIRAI】関連ソング/Songs 【初音ミク/マジカルミライ】,25,,2022-01-20 09:40:05,1:43:8
5,PL-pKPpZ1Q5Nbw0GsDT9V6aGzrQASzWZ6k,【Hatsune Miku/SNOWMIKU】 テーマソング/Theme song 【初音ミク/雪ミク】,10,,2022-01-20 09:22:57,0:41:27
6,PL-pKPpZ1Q5NYrZw1fT6jSLCMwQqwPTewZ,【Kagamine Rin/Len】鏡音リン・レン Happy 14th Birthday #Shorts【鏡音リン・レン】,15,,2021-12-14 08:17:13,0:4:15
7,PL-pKPpZ1Q5NYXIzQjjo-hH6qrJbnb1yKT,【Hatsune Miku】 Demo Songs 【初音ミク】,52,,2017-11-02 02:47:15,1:47:23
8,PL-pKPpZ1Q5NYdsmuLrHLFJg1eL6bXgNDx,【Hatsune Miku】 GSR 【初音ミク】,21,,2017-11-02 01:13:49,1:15:28
9,PL-pKPpZ1Q5NZqi-pHqc2zKXk8GG3U2fkW,【Hatsune Miku】 CONCERT 【初音ミク】,36,,2013-06-13 08:22:49,2:50:2


### Lista de reproducción con su video de más Likes.
Ahora ya tenemos todas las listas de producción pero ¿Cuál de todos esos videos que conforman a la lista es el más popular?

In [25]:
#Vamos a definir las dos funciones faltantes para obtener este resultado.
#La primera nos hará la petición por cada video de sus estadísticas y propiedades de
#estilo o dieseño. La segundo será para obtener los likes en los videos.

#Estadisticas por cada video
def stadistics_video(id_videos):
    video_request  = youtube.videos().list(
        part='statistics,snippet',
        id=','.join(id_videos)
    )
    video_data = video_request.execute()
    return video_data
#Solucion por decorador
def likesvideos_per_playlist(function):
    video_view = []#Tiempo de cada video por cada lista de reproduccion.
    for item in function['items']:
        likes = item['statistics']['likeCount'] #Likes de videos
        video_id = item['id'] #Id del video
        name_video = item['snippet']['title'] #Titulo el video
        yt_link = f'https://youtu.be/{video_id}'#Link del video 
        video_view.append({'title':name_video, 
                            'likeCount':int(likes),
                           'url':yt_link}) #Diccionario donde se almacen los datos anteriores
    video_view.sort(key=lambda vid:vid['likeCount'],reverse=True) #Este método lo que hacemos es ordenarlos de mayor a menor todos sus elementos
    return video_view #Retornamos el diccionario con los videos ordenados con mayores likes por cada lista de reproducción

In [26]:
pl1_staditic  = stadistics_video(rangePlayList[0])
view_plst = likesvideos_per_playlist(pl1_staditic)

In [27]:
#Si nos damos cuenta de algo, es que como regrese un diccionario podemos jugar
#con los indices obteniendo los likes, el link y titulo como podemos ver aquí
view_plst[0]['likeCount'],view_plst[0]['url'],view_plst[0]['title']

(137463, 'https://youtu.be/gdssxLx7ofs', 'STUDY WITH MIKU - part1 -')

In [28]:
#Vamos organizarlo de una manera mejor y finalmente que quede indexado en nuetro dataframe
def mostPopularSongbyPL(listasCanal):
    name_popularvid = [] #Lista de los videos más populares
    liked_list = [] #Lista de likes
    url_list = [] #Urls de los videos
    noneHit = None #En el caso que no hubiera información de ello
    for i in listasCanal:
        staditica = stadistics_video(i) #Llamamos la función de arriba
        try: #Si hay elementos disponibles llenamos las listas 
            liked = likesvideos_per_playlist(staditica) 
            name_popularvid.append(liked[0]['title'])
            liked_list.append(liked[0]['likeCount'])
            url_list.append(liked[0]['url'])
        except:# En el caso que no hubiera un elemento(s) para tratar ese error se hará listas nulas como se observará
            name_popularvid.append(noneHit)
            liked_list.append(noneHit)
            url_list.append(noneHit)
    return [name_popularvid,liked_list,url_list] #Regresamos los nombres,likes y las urls de los videos
mostPopularSongbyPL(rangePlayList)

[['STUDY WITH MIKU - part1 -',
  'Yukopi - 寝起きヤシの木 (feat.歌愛ユキ)',
  '【MV】Lucky☆Orb feat. Hatsune Miku by emon(Tes.)  / ラッキー☆オーブ feat. 初音ミク by emon(Tes.) 【MIKU EXPO 5th】',
  "Giga - '劣等上等'(BRING IT ON) ft.鏡音リン・レン【MV】",
  "Giga - '劣等上等'(BRING IT ON) ft.鏡音リン・レン【MV】",
  '【初音ミク】スターナイトスノウ【オリジナルMV】',
  '#14 今年で設定年齢と同じ14歳を迎える「鏡音リン・レン Happy 14th Birthday」企画🔥2人の14年を1年ずつ振り返る、作家さんによる特別なイラストを連続でお届け✨ #Shorts',
  '初音ミクNTプロトタイプ版・歌声デモンストレーション（※ショート楽曲：3曲が流れます）',
  None,
  '【Hatsune Miku】 World is Mine / ryo（supercell）【初音ミク】',
  '【初音ミク】成人の日お祝いメッセージ',
  "Mikumentary Episode 1: Everybody's Voice / Icon",
  '初音天地開闢神話 / cosMo@暴走P feat. 初音ミク【マジカルミライ 2021】'],
 [137463,
  278962,
  322932,
  724949,
  724949,
  141164,
  19607,
  12470,
  None,
  304670,
  29276,
  1628,
  102322],
 ['https://youtu.be/gdssxLx7ofs',
  'https://youtu.be/FSXsAt7z_RU',
  'https://youtu.be/AufydOsiD6M',
  'https://youtu.be/oEkGC2HV7rc',
  'https://youtu.be/oEkGC2HV7rc',
  'https://youtu.be/ZuT3xYLW7vA',
  'https://youtu.be/0_4stWg5Q_

In [32]:
#Para visualizar los links se puedan dar click a que hacer una pequeña modificacion
#Pero primero indexamos los elementos al dataFrame que teníamos
df['Nombre de Hit'],df['Likes'],df['UrlHit'] = mostPopularSongbyPL(rangePlayList)[0],mostPopularSongbyPL(rangePlayList)[1],mostPopularSongbyPL(rangePlayList)[2]
df = df.dropna()
display(HTML(df.to_html(escape=False,render_links=True,formatters=dict(Uri=a_img_tg)))) #render_links = True lo permite.

,Ids de playlists,Titulos de playlist,Numero de canciones,Uri,Fecha publicacion,Duracion,Nombre de Hit,Likes,UrlHit
0,PL-pKPpZ1Q5NacsYBHSI5wyUQlsV8HhKqL,STUDY WITH MIKU,2,,2023-04-06 04:18:37,3:0:2,STUDY WITH MIKU - part1 -,137463.0,https://youtu.be/gdssxLx7ofs
1,PL-pKPpZ1Q5NYK6AN_5lybs6xc5LlD5Pwj,おすすめのニューリリース/Recommended New Releases,16,,2022-10-21 01:31:46,0:52:49,Yukopi - 寝起きヤシの木 (feat.歌愛ユキ),278964.0,https://youtu.be/FSXsAt7z_RU
2,PL-pKPpZ1Q5Nam8i_mRK4Ky0t0-ihdlE2U,【Hatsune Miku/MIKU EXPO】関連ソング/Songs【初音ミク/MIKU EXPO】,18,,2022-02-08 02:56:45,1:13:42,【MV】Lucky☆Orb feat. Hatsune Miku by emon(Tes.) / ラッキー☆オーブ feat. 初音ミク by emon(Tes.) 【MIKU EXPO 5th】,322932.0,https://youtu.be/AufydOsiD6M
3,PL-pKPpZ1Q5Nbr1RmMnQ7hcyRo_CXLAOsy,【Hatsune Miku】アニバーサリーソング/Anniversary song【初音ミク】,18,,2022-01-20 09:42:41,1:9:34,Giga - '劣等上等'(BRING IT ON) ft.鏡音リン・レン【MV】,724949.0,https://youtu.be/oEkGC2HV7rc
4,PL-pKPpZ1Q5NbP6N1MrSkCKTCHooY_Jbm7,【Hatsune Miku/MAGICAL MIRAI】関連ソング/Songs 【初音ミク/マジカルミライ】,25,,2022-01-20 09:40:05,1:43:8,Giga - '劣等上等'(BRING IT ON) ft.鏡音リン・レン【MV】,724949.0,https://youtu.be/oEkGC2HV7rc
5,PL-pKPpZ1Q5Nbw0GsDT9V6aGzrQASzWZ6k,【Hatsune Miku/SNOWMIKU】 テーマソング/Theme song 【初音ミク/雪ミク】,10,,2022-01-20 09:22:57,0:41:27,【初音ミク】スターナイトスノウ【オリジナルMV】,141164.0,https://youtu.be/ZuT3xYLW7vA
6,PL-pKPpZ1Q5NYrZw1fT6jSLCMwQqwPTewZ,【Kagamine Rin/Len】鏡音リン・レン Happy 14th Birthday #Shorts【鏡音リン・レン】,15,,2021-12-14 08:17:13,0:4:15,#14 今年で設定年齢と同じ14歳を迎える「鏡音リン・レン Happy 14th Birthday」企画🔥2人の14年を1年ずつ振り返る、作家さんによる特別なイラストを連続でお届け✨ #Shorts,19607.0,https://youtu.be/0_4stWg5Q_A
7,PL-pKPpZ1Q5NYXIzQjjo-hH6qrJbnb1yKT,【Hatsune Miku】 Demo Songs 【初音ミク】,52,,2017-11-02 02:47:15,1:47:23,初音ミクNTプロトタイプ版・歌声デモンストレーション（※ショート楽曲：3曲が流れます）,12470.0,https://youtu.be/HNVfTmJV3Rw
9,PL-pKPpZ1Q5NZqi-pHqc2zKXk8GG3U2fkW,【Hatsune Miku】 CONCERT 【初音ミク】,36,,2013-06-13 08:22:49,2:50:2,【Hatsune Miku】 World is Mine / ryo（supercell）【初音ミク】,304670.0,https://youtu.be/jhl5afLEKdo
10,PL-pKPpZ1Q5NYbpC_fY9RToOVfVaaH1i-H,【Hatsune Miku】 MIKU Channel 【初音ミク】,87,,2013-02-06 09:17:51,15:47:2,【初音ミク】成人の日お祝いメッセージ,29276.0,https://youtu.be/70LfL8p25Bg
